## Imports

In [39]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import urllib3
import re
from textwrap import shorten

ModuleNotFoundError: No module named 'geopandas'

In [43]:
!curl https://files.pythonhosted.org/packages/9b/59/3b0907ae8128f81db5cd978d95e6cd6aa2b0ce19884833c37f65deaaca44/geopandas-0.9.0.tar.gz > geopandas.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  968k  100  968k    0     0  5959k      0 --:--:-- --:--:-- --:--:-- 6208k


In [56]:
import geopandas

  Using cached pyproj-3.3.1.tar.gz (219 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      Traceback (most recent call last):
        File "/Users/afarence/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 363, in <module>
          main()
        File "/Users/afarence/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 345, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/Users/afarence/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 130, in get_requires_for_build_wheel
          return hook(config_settings)
        File "/private/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/pip-build-env-9

In [3]:
df = pd.read_csv('LA_map_popup_pin - ktown.csv')

In [4]:
df.columns

Index(['full_address', 'developers', 'total_units',
       'affordable_or_extremely_low_income_units', 'description',
       'story_link'],
      dtype='object')

In [38]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('Brickell.kml', driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

NameError: name 'gpd' is not defined

## Image URL collector

In [8]:
### Drop last two rows of a df. In case there are sums that need clipped
df.drop(df.tail(2).index, inplace=True)

,full_address,developers,total_units,affordable_or_extremely_low_income_units,description,story_link,image_links
0,966 South Vermont Avenue,"Teh Jing Wang, Alchemy Planning",90.0,NaN,"Teh Jing Wang, the property owner, in consulta...",https://therealdeal.com/la/2022/06/23/goodbye-...,
1,626 Kingsley Drive,Jamison Properties,127.0,14.0,The Los Angeles planning department approved a...,https://therealdeal.com/la/2022/06/06/jamisons...,
2,3377 West Olympic Boulevard,Bando Dela Corp,153.0,NaN,Bando Dela Corp propsed to build a seven-story...,https://therealdeal.com/la/2022/04/27/k-town-d...,
3,2306 Beverly Boulevard,Leo Lee,42.0,4.0,"Leo Lee proposed to build an eight-story, 42-u...",https://therealdeal.com/la/2022/04/22/develope...,
4,636 South Berendo Street,Berendo,343.0,38.0,"Westchester-based Berendo proposed a 22-story,...",https://therealdeal.com/la/2022/03/23/develope...,
5,430 South Westmoreland Avenue,Villas at Westmoreland LLC,45.0,26.0,Larchmont-based Westmoreland LLC proposed plan...,https://therealdeal.com/la/2021/12/16/proposed...,
6,West Seventh Street,Bridge Housing,95.0,95.0,San Francisco-based Bridge Housing filed plans...,https://therealdeal.com/la/2021/11/29/nonprofi...,
7,550 Shatto Place,"Townline, Former Development and Urban Offerings",367.0,42.0,"City council unanimously approved a 40-story, ...",https://therealdeal.com/la/2021/10/26/koreatow...,
8,603 South Mariposa Avenue,"Housing Diversity Corporation, STS Construction",92.0,NaN,Two Seattle based developers have partnered on...,https://therealdeal.com/la/2021/11/03/seattle-...,
9,728 South Manhattan Place,ELK Development,728.0,6.0,ELK Development filed plans to build a 60-unit...,https://therealdeal.com/la/2021/09/22/elk-deve...,


In [10]:
df['image_links'] = ''
links = []
for url in df.story_link:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    images = soup.find_all('img')
    counter = 0
    for image in images:
        counter += 1
        if 'jpg' in image['src']:
            links.append(image['src'])
            if counter == len(images):
                links.append('no image found')
            break

df['image_links'] = links

## Article link formatter

In [11]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/la/2022/06/23...
1    <a href="https://therealdeal.com/la/2022/06/06...
2    <a href="https://therealdeal.com/la/2022/04/27...
3    <a href="https://therealdeal.com/la/2022/04/22...
4    <a href="https://therealdeal.com/la/2022/03/23...
5    <a href="https://therealdeal.com/la/2021/12/16...
6    <a href="https://therealdeal.com/la/2021/11/29...
7    <a href="https://therealdeal.com/la/2021/10/26...
8    <a href="https://therealdeal.com/la/2021/11/03...
9    <a href="https://therealdeal.com/la/2021/09/22...
Name: description_link, dtype: object

In [28]:
df.story_link[0]

'https://therealdeal.com/la/2022/06/23/goodbye-dragon-hello-housing-yet-another-apartment-complex-planned-for-koreatown/'

## Google Maps API Geolocater Setup

In [12]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [14]:
df['geo_address'] = df['full_address'] + ', Los Angeles, CA'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## HTML popup formatter

In [34]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


## Map Maker

In [33]:
df.description_link

0    <a href="https://therealdeal.com/la/2022/06/23...
1    <a href="https://therealdeal.com/la/2022/06/06...
2    <a href="https://therealdeal.com/la/2022/04/27...
3    <a href="https://therealdeal.com/la/2022/04/22...
4    <a href="https://therealdeal.com/la/2022/03/23...
5    <a href="https://therealdeal.com/la/2021/12/16...
6    <a href="https://therealdeal.com/la/2021/11/29...
7    <a href="https://therealdeal.com/la/2021/10/26...
8    <a href="https://therealdeal.com/la/2021/11/03...
9    <a href="https://therealdeal.com/la/2021/09/22...
Name: description_link, dtype: object

In [36]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Koreatown')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [37]:
m.save('index.html')